<a href="https://colab.research.google.com/github/oughtinc/ergo/blob/notebooks-readme/prediction-dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Installation:

In [0]:
!pip install --quiet poetry  # Fixes https://github.com/python-poetry/poetry/issues/532
!pip install --quiet git+https://github.com/oughtinc/ergo.git@dashboardv3
!pip install --quiet pendulum seaborn

     |████████████████████████████████| 225kB 2.8MB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 112kB 9.1MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 2.7MB 9.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 522kB 8.5MB/s 
     |████████████████████████████████| 153kB 50.5MB/s 
     |████████████████████████████████| 491kB 50.7MB/s 
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.2 which is incompatible.


Imports:

In [0]:
import ergo
import pendulum
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
%load_ext google.colab.data_table

# Dashboard

In [0]:
metaculus = ergo.Metaculus(username="ought", password="", api_domain="pandemic")

## Prediction results

In [0]:
results = metaculus.get_prediction_results()

In [0]:
results

,time,prediction,resolution,score,question_name
0,1.585643e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.46856537719...",0.342983,-2.370854,[short fuse] How many total confirmed deaths o...
1,1.585722e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.46583502617...",0.342983,-2.234884,[short fuse] How many total confirmed deaths o...
2,1.585643e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.23357114507...",0.220000,2.412762,What will be the US unemployment rate for Marc...
3,1.585722e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.23147780523...",0.220000,2.444497,What will be the US unemployment rate for Marc...
4,1.585806e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.23201019054...",0.220000,2.436899,What will be the US unemployment rate for Marc...
...,...,...,...,...,...
72,1.585188e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.79875500219...",0.748706,1.252148,How many total confirmed cases of novel corona...
73,1.585198e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.78836661923...",0.748706,1.671045,How many total confirmed cases of novel corona...
74,1.585292e+09,"{'d': [{'s': 0.02, 'w': 1, 'x0': 0.78542704801...",0.748706,1.780072,How many total confirmed cases of novel corona...
75,1.584666e+09,"{'d': [{'s': 0.0607, 'w': 1, 'x0': 0.4894, 'lo...",0.474481,1.400453,How many cumulative confirmed cases of coronav...


For continuous questions, higher scores are better.

In [0]:
results.groupby("question_name").mean()[["score", "resolution", "time"]]

,score,resolution,time
question_name,,,
How many COVID-2019 cases in South Korea will be confirmed on March the 27th?,0.438638,0.427635,1.584666e+09
How many COVID-2019 cases in the United Kingdom will be confirmed on March the 27th?,1.058132,0.676471,1.584840e+09
"How many cumulative confirmed cases of coronavirus in San Francisco will be reported by Johns Hopkins CSSE on April 1st, 2020?",1.400453,0.474481,1.584666e+09
"How many cumulative confirmed cases of coronavirus in the entire Bay Area will be reported by Johns Hopkins CSSE on April 1st, 2020?",1.789178,0.049120,1.584666e+09
How many total confirmed cases of novel coronavirus will be reported in the WHO African region by March 27?,0.326747,0.241824,1.585016e+09
How many total confirmed cases of novel coronavirus will be reported in the WHO East Mediterranean region by March 27?,1.352250,0.748706,1.585080e+09
How many total confirmed cases of novel coronavirus will be reported in the WHO European region by March 27?,-1.144452,0.491485,1.585081e+09
How many total confirmed cases of novel coronavirus will be reported in the WHO Region of the Americas by March 27?,0.928360,0.381844,1.585080e+09
How many total confirmed cases of novel coronavirus will be reported in the WHO South-East Asia Region by March 27?,2.294402,0.093433,1.585080e+09


## Resolved questions

In [0]:
resolved_predictions = metaculus.make_questions_df(metaculus.get_questions_json(question_status="resolved", player_status="predicted", pages=9999))

See https://pandemic.metaculus.com/questions/?order_by=-resolve_time&status=resolved&guessed_by=112004

In [0]:
resolved_predictions[["title", "resolve_time", "i_created", "url", "id"]]

,title,resolve_time,i_created,url,id
0,[short fuse] How many total confirmed deaths o...,2020-04-02 04:00:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3934
1,What will be the US unemployment rate for Marc...,2020-04-03 07:17:00+00:00,True,https://pandemic.metaculus.com/api2/questions/...,3922
2,Will Florida go under lockdown between 2020-03...,2020-04-01 17:38:00+00:00,True,https://pandemic.metaculus.com/api2/questions/...,3926
3,How many total confirmed cases of novel corona...,2020-03-27 09:00:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3705
4,How many total confirmed cases of novel corona...,2020-03-27 09:00:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3706
5,How many total confirmed cases of novel corona...,2020-03-27 21:37:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3712
6,How many total confirmed cases of novel corona...,2020-03-27 09:00:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3713
7,How many total confirmed cases of novel corona...,2020-03-27 21:43:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3711
8,How many COVID-2019 cases in the United Kingdo...,2020-03-27 09:00:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3761
9,How many cumulative confirmed cases of coronav...,2020-04-01 19:00:00+00:00,False,https://pandemic.metaculus.com/api2/questions/...,3740


## Open questions

In [0]:
open = metaculus.make_questions_df(metaculus.get_questions_json(question_status="open", player_status="any", pages=9999))

In [0]:
open

,url,page_url,id,author,title,status,resolution,created_time,publish_time,close_time,resolve_time,possibilities,can_use_powers,last_activity_time,activity,comment_count,votes,prediction_timeseries,comment_count_snapshot,user_vote,user_community_vis,my_predictions,i_created,i_predicted
0,https://pandemic.metaculus.com/api2/questions/...,/questions/4042/what-percentage-of-individuals...,4042,104761,What percentage of individuals that have previ...,A,None,2020-04-06 23:07:06.109294+00:00,2020-04-09 22:00:00+00:00,2020-10-08 04:00:00+00:00,2022-01-01 05:55:00+00:00,"{'low': False, 'high': False, 'type': 'continu...",True,2020-04-06T23:07:06.109298Z,0.306009,0,1,[],NaN,0,NaN,None,False,False
1,https://pandemic.metaculus.com/api2/questions/...,/questions/4041/what-percentage-of-individuals...,4041,104761,What percentage of individuals that have previ...,A,None,2020-04-06 22:59:29.687709+00:00,2020-04-09 22:00:00+00:00,2020-10-08 04:00:00+00:00,2022-01-01 05:55:00+00:00,"{'low': False, 'high': False, 'type': 'continu...",True,2020-04-06T22:59:29.687718Z,0.303776,0,2,[],NaN,0,NaN,None,False,False
2,https://pandemic.metaculus.com/api2/questions/...,/questions/4039/will-advamed-member-companies-...,4039,107253,Will AdvaMed member companies succeed in manuf...,A,None,2020-04-06 19:31:41.980870+00:00,2020-04-09 06:00:00+00:00,2020-06-05 18:59:00+00:00,2020-07-05 18:59:00+00:00,{'type': 'binary'},True,2020-04-09T21:47:52.675024Z,0.248700,0,3,"[{'t': 1586440034.1283514, 'distribution': {'a...",NaN,0,NaN,None,False,False
3,https://pandemic.metaculus.com/api2/questions/...,/questions/4030/how-many-of-the-twelve-states-...,4030,110540,How many of the twelve states hit hardest by C...,A,None,2020-04-05 09:33:42.116508+00:00,2020-04-09 06:00:00+00:00,2020-07-01 09:27:00+00:00,2020-08-20 09:27:00+00:00,"{'low': False, 'high': False, 'type': 'continu...",True,2020-04-09T21:46:17.305576Z,3.004425,8,0,"[{'t': 1586415517.5350573, 'distribution': {'s...",NaN,0,NaN,None,False,False
4,https://pandemic.metaculus.com/api2/questions/...,/questions/4018/will-a-us-court-rule-fox-corpo...,4018,110540,Will a US Court rule Fox Corporation or its ow...,A,None,2020-04-04 01:34:59.494271+00:00,2020-04-09 04:00:00+00:00,2021-07-07 01:30:00+00:00,2023-01-06 02:30:00+00:00,{'type': 'binary'},True,2020-04-09T21:49:30.364287Z,2.769037,14,3,"[{'t': 1586409356.9204166, 'distribution': {'a...",NaN,0,NaN,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,https://pandemic.metaculus.com/api2/questions/...,/questions/3721/how-many-total-covid-2019-case...,3721,104761,How many total COVID-2019 cases in the USA wil...,A,None,2020-02-22 15:03:59.979262+00:00,2020-02-24 00:00:00+00:00,2020-04-24 23:00:00+00:00,2020-04-27 10:00:00+00:00,"{'low': 'tail', 'high': 'tail', 'type': 'conti...",True,2020-04-09T20:35:41.774391Z,2.669863,86,34,"[{'t': 1582572817.3445907, 'distribution': {'s...",70.0,0,0.0,None,False,False
101,https://pandemic.metaculus.com/api2/questions/...,/questions/3682/before-the-end-of-2024-will-at...,3682,104761,Before the end of 2024 will at least two publi...,A,None,2020-02-17 00:28:15.701750+00:00,2020-02-19 00:00:00+00:00,2024-01-01 00:00:00+00:00,2028-01-01 00:00:00+00:00,{'type': 'binary'},True,2020-04-09T16:38:48.920197Z,0.310069,13,11,"[{'t': 1582130952.6967824, 'distribution': {'a...",NaN,0,NaN,None,False,False
102,https://pandemic.metaculus.com/api2/questions/...,/questions/3655/will-a-vaccine-targeted-at-the...,3655,105951,Will a vaccine targeted at the 2019 novel coro...,A,None,2020-02-11 16:47:52.867810+00:00,2020-02-13 23:00:00+00:00,2020-07-31 21:59:00+00:00,2020-07-31 22:00:00+00:00,{'type': 'binary'},True,2020-04-09T08:20:10.271249Z,0.217326,63,32,"[{'t': 1581638183.2147524, 'distribution': {'a...",NaN,0,NaN,None,False,False
103,https://pandemic.metaculus.com/api2/questions/...,/questions/3530/how-many-people-will-die-as-a-...,3530,101465,How many people will die as a result of th

### Published on or after Mar 25, closing on or before Apr 25

In [0]:
after_mar_25 = open[open["publish_time"] > pendulum.datetime(2020,3,24, tz="America/Los_Angeles")]
before_apr_25 = after_mar_25[after_mar_25["close_time"] < pendulum.datetime(2020,4,26, tz="America/Los_Angeles")]

In [0]:
before_apr_25[["title", "publish_time", "close_time", "i_predicted", "i_created", "page_url", "id"]]

,title,publish_time,close_time,i_predicted,i_created,page_url,id
7,Will the US Strategic National Stockpile suppl...,2020-04-07 06:00:00+00:00,2020-04-25 18:59:00+00:00,False,False,/questions/4020/will-the-us-strategic-national...,4020
11,Will Boris Johnson recover from COVID-19?,2020-04-07 04:00:00+00:00,2020-04-12 10:59:00+00:00,False,False,/questions/4040/will-boris-johnson-recover-fro...,4040
21,How many COVID-19 deaths will be recorded in t...,2020-04-04 04:00:00+00:00,2020-04-15 22:59:00+00:00,True,False,/questions/3996/how-many-covid-19-deaths-will-...,3996
28,What will the CBO estimate to be the cost of t...,2020-04-01 22:00:00+00:00,2020-04-26 06:59:00+00:00,True,True,/questions/3920/what-will-the-cbo-estimate-to-...,3920
30,How many days will the average American spend ...,2020-03-31 22:00:00+00:00,2020-04-21 06:59:00+00:00,True,True,/questions/3925/how-many-days-will-the-average...,3925
33,What will Washington state’s Department of Rev...,2020-03-28 23:00:00+00:00,2020-04-26 06:59:00+00:00,True,True,/questions/3923/what-will-washington-states-de...,3923
34,What will the Seattle Police Department report...,2020-03-28 23:00:00+00:00,2020-04-26 06:59:00+00:00,True,True,/questions/3924/what-will-the-seattle-police-d...,3924
35,Will the US federal government shut down all n...,2020-03-28 23:00:00+00:00,2020-04-11 06:59:00+00:00,True,True,/questions/3921/will-the-us-federal-government...,3921
36,Will the Emergency Telework Act (S.3561) becom...,2020-03-28 23:00:00+00:00,2020-04-20 06:59:00+00:00,True,True,/questions/3918/will-the-emergency-telework-ac...,3918
38,By May 1 will there be an iOS or Android app t...,2020-03-28 13:28:37.595000+00:00,2020-04-21 07:00:00+00:00,True,False,/questions/3915/by-may-1-will-there-be-an-ios-...,3915


# TODO:

1.   Add date and time of our latest prediction
2. Add everything we'd want to reflect on resolutions of predictions (see https://docs.google.com/document/d/1WfWnvDDUufHe39l2UeWZ8lfrQtEaSR9kH7KC_gtEYiQ/edit# for inspiration)



